In [118]:
from sklearn import tree
from IPython.display import Image
import pydotplus
from pydotplus import graphviz
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
%matplotlib inline

In [119]:
df=pd.read_csv('the-number-of-estimated-eligible-uninsured-people-for-outreach-targeting.csv')
df.dropna()
df.head()

,StateName,StateFIPS,PumaState,PUMA,Counties Within PUMA,uninsured_total,uninsured_percent,Foodstamp,Age0_18,Age19_25,...,EnglishSpeaking,Spanish,Chinese,Korean,Vietnamese,Tagalog,Russian,OtherLang,SpeaksMostCommonOthLang,OtherLanguage
0,Alabama,1,100100,100,"Lauderdale County, Colbert County",18517,15.3,7754.0,445.0,4358,...,17665,852.0,-,NaN,NaN,NaN,NaN,NaN,NaN,**
1,Alabama,1,100200,200,Madison County (part),28109,19.0,8364.0,1506.0,4865,...,27579,491.0,-,NaN,38.0,NaN,NaN,NaN,NaN,**
2,Alabama,1,100300,300,Madison County (part),26568,12.1,6620.0,1147.0,4786,...,25729,839.0,-,NaN,NaN,NaN,NaN,NaN,NaN,**
3,Alabama,1,100400,400,"DeKalb County, Jackson County",21245,20.4,6330.0,6713.0,2582,...,20800,446.0,-,NaN,NaN,NaN,NaN,NaN,NaN,**
4,Alabama,1,100500,500,"Marshall County, Cullman County, Winston County",25654,16.0,8155.0,3627.0,5429,...,24218,1313.0,7,NaN,NaN,24.0,NaN,92.0,NaN,**


In [120]:
df.dtypes

StateName                         object
StateFIPS                          int64
PumaState                          int64
PUMA                               int64
Counties Within PUMA              object
uninsured_total                    int64
uninsured_percent                float64
Foodstamp                        float64
Age0_18                          float64
Age19_25                           int64
Age26_34                         float64
Age35_54                          object
Age55_64                          object
138PctFPL_or_Less                 object
139to400PctFPL                    object
Above400PctFPL                    object
Male                              object
Female                            object
Married                           object
ChildinFamily                     object
Latino                            object
WhiteNonLatino                    object
BlackNonLatino                    object
AsianNonLatino                    object
HawaiiPacIslande

In [129]:
import re
no_hyphens=re.sub('-','','123-45-6789')
columns=['uninsured_total','uninsured_percent','Foodstamp','Age0_18','Age26_34']
Tree_df=df[columns]
def clean_dataset(Tree_df):
    assert isinstance(Tree_df, pd.DataFrame), "Tree_df needs to be a pd.DataFrame"
    Tree_df.dropna(inplace=True)
    indices_to_keep = ~Tree_df.isin([np.nan, np.inf, -np.inf]).any(1)
    return Tree_df[indices_to_keep].astype(np.float64)
clean_dataset(Tree_df)
Tree_df.tail()

/Users/samuelkim/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,uninsured_total,uninsured_percent,Foodstamp,Age0_18,Age26_34
2064,10295,10.7,3780.0,583.0,2922.0
2065,22304,18.2,2569.0,3311.0,3955.0
2066,21554,18.6,5207.0,3298.0,4962.0
2067,23115,16.7,3057.0,3272.0,4587.0
2068,15521,13.6,1147.0,1939.0,3627.0


In [128]:
Tree_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2050 entries, 0 to 2068
Data columns (total 5 columns):
uninsured_total      2050 non-null int64
uninsured_percent    2050 non-null float64
Foodstamp            2050 non-null float64
Age0_18              2050 non-null float64
Age26_34             2050 non-null float64
dtypes: float64(4), int64(1)
memory usage: 96.1 KB


In [127]:
#Tree_df.dropna(subset=['uninsured_total','uninsured_percent','Foodstamp','Age0_18','Age26_34'],how='all',with_inf=True)
Tree_df.replace([np.inf, -np.inf], np.nan)
Tree_df.dropna()

,uninsured_total,uninsured_percent,Foodstamp,Age0_18,Age26_34
0,18517,15.3,7754.0,445.0,4922.0
1,28109,19.0,8364.0,1506.0,8068.0
2,26568,12.1,6620.0,1147.0,6231.0
3,21245,20.4,6330.0,6713.0,2940.0
4,25654,16.0,8155.0,3627.0,3614.0
5,19470,15.6,4905.0,2344.0,4286.0
6,19421,17.8,7781.0,487.0,3161.0
7,27814,15.8,11558.0,3073.0,7066.0
8,16726,20.5,7831.0,1648.0,3564.0
9,16132,19.2,9085.0,1281.0,4385.0


## Decision Tree

In [192]:
Xa=df.drop(['Foodstamp'], 1)
X=Xa
Y=df['Foodstamp']

In [202]:
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from datetime import datetime
from tempfile import TemporaryFile
import os 

regressor_tree = tree.DecisionTreeRegressor(criterion='mse', max_depth=25, max_features=4, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'
)

regressor_tree.fit(X, Y)

DecisionTreeRegressor(criterion='mse', max_depth=25, max_features=4,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [194]:
df=Tree_df
print(df.to_csv(sep=' ', index=False, header=False))

18517 15.3 7754.0 445.0 4922.0
28109 19.0 8364.0 1506.0 8068.0
26568 12.1 6620.0 1147.0 6231.0
21245 20.4 6330.0 6713.0 2940.0
25654 16.0 8155.0 3627.0 3614.0
19470 15.6 4905.0 2344.0 4286.0
19421 17.8 7781.0 487.0 3161.0
27814 15.8 11558.0 3073.0 7066.0
16726 20.5 7831.0 1648.0 3564.0
16132 19.2 9085.0 1281.0 4385.0
11433 8.1 3574.0 1066.0 2808.0
22884 14.3 5101.0 3087.0 6276.0
13013 14.1 5332.0 655.0 3102.0
12616 14.6 3964.0 260.0 2976.0
22499 17.3 9156.0 1354.0 4001.0
19109 19.2 10840.0 2013.0 5065.0
17320 10.0 6700.0 3112.0 3589.0
16024 9.3 5405.0 1786.0 4338.0
15043 14.7 6414.0 825.0 2972.0
18084 16.4 7664.0 2263.0 4523.0
14928 11.7 4532.0 953.0 3280.0
31361 18.7 12992.0 3117.0 6348.0
31436 17.2 15577.0 2998.0 7546.0
23254 16.9 6504.0 2033.0 5808.0
30315 23.6 15422.0 3331.0 5532.0
32763 20.0 9742.0 3670.0 7344.0
33890 17.7 11770.0 2560.0 6253.0
20124 13.0 4153.0 1062.0 1260.0
16861 17.7 5569.0 656.0 5368.0
20125 15.5 8409.0 1578.0 4435.0
22555 17.5 3079.0 4969.0 5182.0
17501 12.4 

In [195]:
from sklearn.model_selection import cross_val_score
tree_score=cross_val_score(regressor_tree, X, Y, cv=4)
print('Mean:', tree_score.mean(), '\nVariance:', tree_score.std()*2)

Mean: 0.18296002230518915 
Variance: 0.43117676303326724


In [198]:
from sklearn import ensemble

forest_regression = ensemble.RandomForestClassifier(criterion='entropy', max_depth=25, max_features=4, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, random_state=None)

forest_score = cross_val_score(forest_regression, X, Y, cv=4)
print('Mean:', forest_score.mean(), '\nVariance:', forest_score.std()*2)

/Users/samuelkim/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


Mean: 0.00040496760259179265 
Variance: 0.0014028489262166931


In [199]:
import time
start = time.time()

regressor_tree = tree.DecisionTreeRegressor(criterion='mse', max_depth=25, max_features=4, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best'
)

tree_score = cross_val_score(regressor_tree,X,Y,cv=5)

end = time.time()
tree_time = end-start
print(tree_time)

0.08124303817749023


In [200]:
start = time.time()

forest_regression = ensemble.RandomForestClassifier(criterion='entropy', max_depth=25, max_features=4, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0.0, random_state=None)

forest_score = cross_val_score(forest_regression, X, Y, cv=4)

end = time.time()
forest_time = end-start

print(forest_time)

/Users/samuelkim/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=4.
  % (min_groups, self.n_splits)), Warning)


16.759828805923462


There is a noticeable difference in time, forest regression takes 16.7 seconds as opposed to decision tree's 0.08124 seconds. 